In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Ruta al video
video_path = "../videos/video1.mp4"  # Cambia al video deseado

cap = cv2.VideoCapture(video_path)

frame_count = 0
prev_frame = None
num_prev_frame = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Convertir a escala de grises
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Procesar solo en múltiplos de 30
    if frame_count % 30 == 0:
        if prev_frame is not None:
            # Calcular diferencias
            diff = cv2.absdiff(prev_frame, frame_gray)
            
            # Umbralización
            _, diff_thresh = cv2.threshold(diff, 70, 255, cv2.THRESH_BINARY)
            
            # Mostrar resultados
            plt.figure(figsize=(15, 10))
            plt.subplot(1, 3, 1)
            plt.title(f"Frame Base {num_prev_frame}")
            plt.imshow(prev_frame, cmap="gray")
            plt.axis("off")

            plt.subplot(1, 3, 2)
            plt.title(f"Frame Actual {frame_count}")
            plt.imshow(frame_gray, cmap="gray")
            plt.axis("off")

            plt.subplot(1, 3, 3)
            plt.title("Diferencia (Umbralizada)")
            plt.imshow(diff_thresh, cmap="gray")
            plt.axis("off")
            plt.show()

        # Actualizar el frame base
        prev_frame = frame_gray
        num_prev_frame = frame_count

cap.release()



In [ ]:
import cv2
import numpy as np

# Ruta al video
video_path = "../videos/video1.mp4"  # Cambia al video deseado

cap = cv2.VideoCapture(video_path)

frame_count = 0
prev_frame = None

# Crear un objeto para detección de movimiento (fondo acumulado)
fgbg = cv2.createBackgroundSubtractorMOG2()

# Definir un tamaño más pequeño para la ventana
resize_width = 640  # Ancho deseado
resize_height = 360  # Alto deseado

# Reducir la resolución del video para mejorar el rendimiento
downscale_factor = 0.5  # Reducción al 50% del tamaño original

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    # Reducir la resolución globalmente
    frame_resized = cv2.resize(frame, None, fx=downscale_factor, fy=downscale_factor)
    
    # Convertir a escala de grises
    frame_gray = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)

    # Aplicar sustracción de fondo (opcional pero útil para entornos reales)
    fgmask = fgbg.apply(frame_gray)
    
    # Procesar solo en múltiplos de 5 frames para reducir la carga de trabajo
    if frame_count % 30 == 0:
        if prev_frame is not None:
            # Calcular diferencia entre el frame actual y el anterior
            diff = cv2.absdiff(prev_frame, frame_gray)
            
            # Umbralización para obtener la diferencia binaria
            _, diff_thresh = cv2.threshold(diff, 99, 255, cv2.THRESH_BINARY)

            # Aplicar dilatación para ampliar las áreas detectadas (mejorar contornos)
            diff_thresh = cv2.dilate(diff_thresh, None, iterations=2)

            # Contar los píxeles cambiados (es decir, detectar el movimiento)
            non_zero_count = cv2.countNonZero(diff_thresh)

            # Si el número de píxeles cambiados es mayor que un umbral, se detecta movimiento
            if non_zero_count > 500:
                print(f"Movimiento detectado en el frame {frame_count}")

        # Actualizar el frame base
        prev_frame = frame_gray

    # Redimensionar las imágenes a un tamaño más pequeño para su visualización
    diff_resized = cv2.resize(diff_thresh, (resize_width, resize_height))

    # Mostrar el video y las diferencias
    cv2.imshow('Video Original', frame_resized)
    cv2.imshow('Diferencias Umbralizadas', diff_resized)

    # Salir con la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
